In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Kobe Bryant  shot selection

Cсылка на соревнование: https://www.kaggle.com/c/kobe-bryant-shot-selection

Using 20 years of data on Kobe's swishes and misses, can you predict which shots will find the bottom of the net? This competition is well suited for practicing classification basics, feature engineering, and time series analysis. Practice got Kobe an eight-figure contract and 5 championship rings. What will it get you?

This data contains the location and circumstances of every field goal attempted by Kobe Bryant took during his 20-year career. Your task is to predict whether the basket went in (shot_made_flag).

We have removed 5000 of the shot_made_flags (represented as missing values in the csv file). These are the test set shots for which you must submit a prediction. You are provided a sample submission file with the correct shot_ids needed for a valid prediction.

In [62]:
data = pd.read_csv('data/Kobe.csv')

In [63]:
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [64]:
target = 'shot_made_flag'

In [65]:
nona = data[pd.notnull(data[target])]

In [66]:
nona.shape

(25697, 25)

In [67]:
nona.describe()

,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
count,25697.000000,2.569700e+04,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,2.569700e+04,25697.000000
mean,249.348679,2.474109e+07,33.953043,7.148422,91.257345,-118.262652,4.886796,2.520800,0.146243,28.311554,13.457096,0.446161,1.610613e+09,15328.166946
std,149.778520,7.738108e+06,0.088152,110.073147,88.152106,0.110073,3.452475,1.151626,0.353356,17.523392,9.388725,0.497103,0.000000e+00,8860.462397
min,2.000000,2.000001e+07,33.253300,-250.000000,-44.000000,-118.519800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,2.000000
25%,111.000000,2.050006e+07,33.884300,-67.000000,4.000000,-118.336800,2.000000,1.000000,0.000000,13.000000,5.000000,0.000000,1.610613e+09,7646.000000
50%,253.000000,2.090034e+07,33.970300,0.000000,74.000000,-118.269800,5.000000,3.000000,0.000000,28.000000,15.000000,0.000000,1.610613e+09,15336.000000
75%,367.000000,2.960027e+07,34.040300,94.000000,160.000000,-118.175800,8.000000,3.000000,0.000000,43.000000,21.000000,1.000000,1.610613e+09,22976.000000
max,653.000000,4.990009e+07,34.088300,248.000000,791.000000,-118.021800,11.000000,7.000000,1.000000,59.000000,79.000000,1.000000,1.610613e+09,30697.000000


In [68]:
categorical_columns = [c for c in nona.columns if nona[c].dtype.name == 'object']
numerical_columns   = [c for c in nona.columns if nona[c].dtype.name != 'object']
print (categorical_columns)
print (numerical_columns)

['action_type', 'combined_shot_type', 'season', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'team_name', 'game_date', 'matchup', 'opponent']
['game_event_id', 'game_id', 'lat', 'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs', 'seconds_remaining', 'shot_distance', 'shot_made_flag', 'team_id', 'shot_id']


In [69]:
nona[categorical_columns].describe()

,action_type,combined_shot_type,season,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_name,game_date,matchup,opponent
count,25697,25697,25697,25697,25697,25697,25697,25697,25697,25697,25697
unique,55,6,20,2,6,7,5,1,1558,74,33
top,Jump Shot,Jump Shot,2005-06,2PT Field Goal,Center(C),Mid-Range,Less Than 8 ft.,Los Angeles Lakers,2016-04-13,LAL @ SAS,SAS
freq,15836,19710,1924,20285,11289,10532,7857,25697,43,853,1638


In [70]:
nona.describe(include=[object])

,action_type,combined_shot_type,season,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_name,game_date,matchup,opponent
count,25697,25697,25697,25697,25697,25697,25697,25697,25697,25697,25697
unique,55,6,20,2,6,7,5,1,1558,74,33
top,Jump Shot,Jump Shot,2005-06,2PT Field Goal,Center(C),Mid-Range,Less Than 8 ft.,Los Angeles Lakers,2016-04-13,LAL @ SAS,SAS
freq,15836,19710,1924,20285,11289,10532,7857,25697,43,853,1638


In [71]:
nona[numerical_columns].describe()

,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
count,25697.000000,2.569700e+04,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,2.569700e+04,25697.000000
mean,249.348679,2.474109e+07,33.953043,7.148422,91.257345,-118.262652,4.886796,2.520800,0.146243,28.311554,13.457096,0.446161,1.610613e+09,15328.166946
std,149.778520,7.738108e+06,0.088152,110.073147,88.152106,0.110073,3.452475,1.151626,0.353356,17.523392,9.388725,0.497103,0.000000e+00,8860.462397
min,2.000000,2.000001e+07,33.253300,-250.000000,-44.000000,-118.519800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,2.000000
25%,111.000000,2.050006e+07,33.884300,-67.000000,4.000000,-118.336800,2.000000,1.000000,0.000000,13.000000,5.000000,0.000000,1.610613e+09,7646.000000
50%,253.000000,2.090034e+07,33.970300,0.000000,74.000000,-118.269800,5.000000,3.000000,0.000000,28.000000,15.000000,0.000000,1.610613e+09,15336.000000
75%,367.000000,2.960027e+07,34.040300,94.000000,160.000000,-118.175800,8.000000,3.000000,0.000000,43.000000,21.000000,1.000000,1.610613e+09,22976.000000
max,653.000000,4.990009e+07,34.088300,248.000000,791.000000,-118.021800,11.000000,7.000000,1.000000,59.000000,79.000000,1.000000,1.610613e+09,30697.000000


In [72]:
for c in categorical_columns:
    print (nona[c].unique())

['Jump Shot' 'Driving Dunk Shot' 'Layup Shot' 'Running Jump Shot'
 'Reverse Dunk Shot' 'Slam Dunk Shot' 'Driving Layup Shot'
 'Turnaround Jump Shot' 'Reverse Layup Shot' 'Tip Shot'
 'Running Hook Shot' 'Alley Oop Dunk Shot' 'Dunk Shot'
 'Alley Oop Layup shot' 'Running Dunk Shot' 'Driving Finger Roll Shot'
 'Running Layup Shot' 'Finger Roll Shot' 'Fadeaway Jump Shot'
 'Follow Up Dunk Shot' 'Hook Shot' 'Turnaround Hook Shot' 'Jump Hook Shot'
 'Running Finger Roll Shot' 'Jump Bank Shot' 'Turnaround Finger Roll Shot'
 'Hook Bank Shot' 'Driving Hook Shot' 'Running Tip Shot'
 'Running Reverse Layup Shot' 'Driving Finger Roll Layup Shot'
 'Fadeaway Bank shot' 'Pullup Jump shot' 'Finger Roll Layup Shot'
 'Turnaround Fadeaway shot' 'Driving Reverse Layup Shot'
 'Driving Slam Dunk Shot' 'Step Back Jump shot' 'Turnaround Bank shot'
 'Reverse Slam Dunk Shot' 'Floating Jump shot' 'Putback Slam Dunk Shot'
 'Running Bank shot' 'Driving Bank shot' 'Driving Jump shot'
 'Putback Layup Shot' 'Putback Dun

**Исключаем повторяющиеся категории**

In [73]:
drops = ['shot_id', 'team_id', 'team_name', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', \
         'matchup', 'lon', 'lat', 'seconds_remaining', 'minutes_remaining', \
         'shot_distance', 'loc_x', 'loc_y', 'game_event_id', 'game_id', 'game_date']
for drop in drops:
    nona = nona.drop(drop, 1)

In [74]:
nona.shape

(25697, 8)

In [84]:
nona.head()

,period,playoffs,shot_made_flag,action_type_Alley Oop Dunk Shot,action_type_Alley Oop Layup shot,action_type_Cutting Layup Shot,action_type_Driving Bank shot,action_type_Driving Dunk Shot,action_type_Driving Finger Roll Layup Shot,action_type_Driving Finger Roll Shot,...,opponent_PHI,opponent_PHX,opponent_POR,opponent_SAC,opponent_SAS,opponent_SEA,opponent_TOR,opponent_UTA,opponent_VAN,opponent_WAS
1,1,0,0.0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,1.0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0.0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2,0,1.0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
5,3,0,0.0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


**Делаем нормализацию категориальных данных**

In [80]:
categorical_columns = [c for c in nona.columns if nona[c].dtype.name == 'object']

In [81]:
for var in categorical_columns:
    nona = pd.concat([nona, pd.get_dummies(nona[var], prefix=var)], 1)
    nona = nona.drop(var, 1)

In [82]:
numerical_columns   = [c for c in nona.columns if nona[c].dtype.name != 'object']


In [83]:
print(numerical_columns)

['period', 'playoffs', 'shot_made_flag', 'action_type_Alley Oop Dunk Shot', 'action_type_Alley Oop Layup shot', 'action_type_Cutting Layup Shot', 'action_type_Driving Bank shot', 'action_type_Driving Dunk Shot', 'action_type_Driving Finger Roll Layup Shot', 'action_type_Driving Finger Roll Shot', 'action_type_Driving Floating Bank Jump Shot', 'action_type_Driving Floating Jump Shot', 'action_type_Driving Hook Shot', 'action_type_Driving Jump shot', 'action_type_Driving Layup Shot', 'action_type_Driving Reverse Layup Shot', 'action_type_Driving Slam Dunk Shot', 'action_type_Dunk Shot', 'action_type_Fadeaway Bank shot', 'action_type_Fadeaway Jump Shot', 'action_type_Finger Roll Layup Shot', 'action_type_Finger Roll Shot', 'action_type_Floating Jump shot', 'action_type_Follow Up Dunk Shot', 'action_type_Hook Bank Shot', 'action_type_Hook Shot', 'action_type_Jump Bank Shot', 'action_type_Jump Hook Shot', 'action_type_Jump Shot', 'action_type_Layup Shot', 'action_type_Pullup Bank shot', 'ac

**Линейная регрессия**

In [87]:
del numerical_columns[2]

In [88]:
print(numerical_columns)

['period', 'playoffs', 'action_type_Alley Oop Layup shot', 'action_type_Cutting Layup Shot', 'action_type_Driving Bank shot', 'action_type_Driving Dunk Shot', 'action_type_Driving Finger Roll Layup Shot', 'action_type_Driving Finger Roll Shot', 'action_type_Driving Floating Bank Jump Shot', 'action_type_Driving Floating Jump Shot', 'action_type_Driving Hook Shot', 'action_type_Driving Jump shot', 'action_type_Driving Layup Shot', 'action_type_Driving Reverse Layup Shot', 'action_type_Driving Slam Dunk Shot', 'action_type_Dunk Shot', 'action_type_Fadeaway Bank shot', 'action_type_Fadeaway Jump Shot', 'action_type_Finger Roll Layup Shot', 'action_type_Finger Roll Shot', 'action_type_Floating Jump shot', 'action_type_Follow Up Dunk Shot', 'action_type_Hook Bank Shot', 'action_type_Hook Shot', 'action_type_Jump Bank Shot', 'action_type_Jump Hook Shot', 'action_type_Jump Shot', 'action_type_Layup Shot', 'action_type_Pullup Bank shot', 'action_type_Pullup Jump shot', 'action_type_Putback Dun

In [89]:
from sklearn.linear_model import LinearRegression

def linear_regression(nona, power):
    # predictors:
    predictors=numerical_columns
    
    # fit
    linreg = LinearRegression(normalize=True)
    linreg.fit(nona[predictors], nona['shot_made_flag'])
    y_pred = linreg.predict(nona[predictors])
    

    
    # Return the result in pre-defined format
    rss = sum((y_pred - nona['shot_made_flag']) ** 2)
    ret = [rss]
    ret.extend([linreg.intercept_])
    ret.extend(linreg.coef_)
    return ret

In [ ]:
#Initialize a dataframe to store the results:
col = ['rss', 'intercept'] + ['coef_x_%d' % i for i in range(1, 16)]
ind = ['model_pow_%d' % i for i in range(1, 16)]
coef_matrix_simple = pd.DataFrame(index=ind, columns=col)

#Iterate through all powers and assimilate results
for i in range(1, 16):
    coef_matrix_simple.iloc[i - 1, 0:i + 2] = linear_regression(
        data,
        power=i,
        models_to_plot=models_to_plot
    )

In [90]:
pd.options.display.float_format = '{:,.2g}'.format
coef_matrix_simple

NameError: name 'coef_matrix_simple' is not defined

**Задания:**

1. Провести анализ данных. Много хороших примеров анализа можно посмотреть здесь https://www.kaggle.com/c/kobe-bryant-shot-selection/kernels
2. Подготовить фичи для обучения модели - нагенерить признаков, обработать пропущенные значения, проверить на возможные выбросы, обработать категориальные признаки и др.
3. Обучить линейную модель, Lasso, Ridge на тех же признаках - построить сравнительную таблицу коэффициентов, сделать заключения о том, как меняется величина коэффициентов, какие зануляются. Посчитать RSS

**Дополнительно**
4. Сравнить результаты на тестовом наборе данных - сделать train_test_split в самом начале, подготовить переменные, сравнитьрезультаты работы классификаторов (те же 3), метрика ROC AUC
5. Построить PCA на подготовленных признаках, посмотреть, какие компоненты составляют наибольшую часть дисперсии целевой переменной